In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
start_time = time.time()

# Used for loading csv files into excel
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import os
import glob

In [ ]:
df = pd.read_excel(r"/Users/Sean/Desktop/Analysis of Data.xlsx")


In [ ]:
# Dropped anomalous readings
above_ground = df[(df["Above Ground"]=="Yes") & (df["Distance Travelled (m)"] > 1000)]
below_ground = df[(df["Above Ground"]=="No") & (df["Distance Travelled (m)"] > 1000)]
# Note: Anomalous readings are from 61-67

In [ ]:
# Initialise variables

# Mean Acceleration
below_ground_y = below_ground["Mean Acceleration (ms^-2)"]
above_ground_y = above_ground["Mean Acceleration (ms^-2)"]

# Total Distance
below_ground_x = below_ground["Distance Travelled (m)"]
above_ground_x = above_ground["Distance Travelled (m)"]

# Total Time Taken
below_ground_t = below_ground["Total Time Taken (s)"]
above_ground_t = above_ground["Total Time Taken (s)"]

# Max Velocity
below_ground_v = below_ground["Max Velocity (ms^-1)"]
above_ground_v = above_ground["Max Velocity (ms^-1)"]

# Mean Velocity
below_ground_mean_v = below_ground["Distance Travelled (m)"] / below_ground["Total Time Taken (s)"]
above_ground_mean_v = above_ground["Distance Travelled (m)"] / above_ground["Total Time Taken (s)"]

In [ ]:
plt.scatter(below_ground_t, below_ground_y, c='b', label='Below Ground')
plt.scatter(above_ground_t, above_ground_y, c='r', label='Above Ground')
plt.xlabel('Total Time Taken (s)')
plt.ylabel('Mean Acceleration (ms^-2)')
plt.legend(loc='best')
plt.gcf().set_size_inches((10, 10))  
plt.show()

In [ ]:
plt.scatter(below_ground_x, below_ground_v, c='b', label='Below Ground')
plt.scatter(above_ground_x, above_ground_v, c='r', label='Above Ground')
plt.xlabel('Distance Travelled (m)')
plt.ylabel('Max Velocity (ms^-1)')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.scatter(below_ground_x, below_ground_mean_v, c='b', label='Below Ground')
plt.scatter(above_ground_x, above_ground_mean_v, c='r', label='Above Ground')
plt.xlabel('Distance Travelled (m)')
plt.ylabel('Mean Velocity (ms^-1)')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.scatter(below_ground_x, below_ground_t, c='b', label='Below Ground')
plt.scatter(above_ground_x, above_ground_t, c='r', label='Above Ground')
plt.xlabel('Total Distance Travelled (m)')
plt.ylabel('Total Time Taken (s)')
plt.legend(loc='best')
plt.gcf().set_size_inches((10, 10))    
plt.show()

In [ ]:
# To remove samples which still appear slightly anomalous
below_ground_x_2 = below_ground[below_ground["Total Time Taken (s)"] < 200]["Distance Travelled (m)"]
above_ground_x_2 = above_ground[above_ground["Total Time Taken (s)"] < 200]["Distance Travelled (m)"]

In [ ]:
mean_t_below = below_ground_t[below_ground_t < 200].mean()
mean_t_above = above_ground_t[above_ground_t < 200].mean()
median_t_below = below_ground_t[below_ground_t < 200].median()
median_t_above = above_ground_t[above_ground_t < 200].median()

mean_v_below = sum(below_ground_x_2) / sum(below_ground_t[below_ground_t < 200])
mean_v_above = sum(above_ground_x_2) / sum(above_ground_t[above_ground_t < 200])

print(
f'''
Number of samples below ground = {len(below_ground_t[below_ground_t < 200])}
Number of samples above ground = {len(above_ground_t[above_ground_t < 200])}

Mean t (Below) = {mean_t_below}
Median t (Below) = {median_t_below}
Mean v (Below) = {mean_v_below}
Mean t (Above) = {mean_t_above}
Median t (Above) = {median_t_above}
Mean v (Above) = {mean_v_above}
Difference in mean t = {mean_t_above - mean_t_below}
Difference in median t = {median_t_above - median_t_below}
Difference in average v = {mean_v_above - mean_v_below}
''')

In [ ]:
above_ground[above_ground["Distance Travelled (m)"] > 4000]

In [ ]:
below_ground[below_ground["Distance Travelled (m)"] < 1000]